In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import tri, cm
import meshio
from tqdm import tqdm

Definições Geométricas

In [4]:
msh = meshio.read('./mesh/new_cube.msh')

X = msh.points[:,0]
Y = msh.points[:,1]
Z = msh.points[:,2]

In [5]:
IEN = np.array([])
IENBound = np.array([])

for cellBlock in msh.cells:
    if cellBlock.type == 'triangle':
        if IENBound.shape == (0,):
            IENBound = cellBlock.data
        else:
            IENBound = np.concatenate([IENBound, cellBlock.data])
    elif cellBlock.type == 'tetra':
        if IEN.shape == (0,):
            IEN = cellBlock.data
        else:
            IEN = np.concatenate([IEN, cellBlock.data])

In [6]:
Npoints = len(X)
ne = IEN.shape[0]

In [ ]:
# IENboundTypeElem = list(msh.cell_data['gmsh:physical'][0] - 1)
# boundNames = list(msh.field_data.keys())
# IENboundElem = [boundNames[elem] for elem in IENboundTypeElem]
# npoints = len(X)
# ne = IEN.shape[0]

In [ ]:
# # cria lista de nos do contorno
# cc = np.unique(IENbound.reshape(IENbound.size))
# ccName = [[] for i in range( len(X) )]
# for elem in range(0,len(IENbound)):
#  ccName[ IENbound[elem][0] ] = IENboundElem[elem]
#  ccName[ IENbound[elem][1] ] = IENboundElem[elem]
#  ccName[ IENbound[elem][2] ] = IENboundElem[elem]
# # plot malha e pontos de contorno (export para vtk)
# meshio.write_points_cells('malha.vtk',
#  msh.points,
#  msh.cells,)

In [7]:
cc = np.unique(IENBound.reshape(IENBound.size))
meshio.write_points_cells('./mesh/new_cube.vtk',
 msh.points,
 msh.cells,)

Definições do material

In [8]:
rho = 1
cv = 1
alpha = 1

kappa = [1, 1, 1] # kx, ky, kz

Q = -6*rho*cv

In [10]:
#form functions for 2D
def IENfunc3DTet(K, M, IEN):
    for IENelem in tqdm(IEN):
        tet_matrix = np.array([[1, X[IENelem[i]], Y[IENelem[i]] , Z[IENelem[i]]] for i in range(4)])
        tet_volume = np.linalg.det(tet_matrix)/6

        melem = (tet_volume/20)*np.array(([2, 1, 1, 1],
                                          [1, 2, 1, 1],
                                          [1, 1, 2, 1],
                                          [1, 1, 1, 2]))
        
        [a_list, b_list, c_list, d_list] = np.linalg.inv(tet_matrix)
        
        kxelem = kappa[0]*tet_volume*np.array([[b_list[i]*b_list[j] for i in range(4)] for j in range(4)])
        kyelem = kappa[1]*tet_volume*np.array([[c_list[i]*c_list[j] for i in range(4)] for j in range(4)])
        kzelem = kappa[2]*tet_volume*np.array([[d_list[i]*d_list[j] for i in range(4)] for j in range(4)])

        kelem = kxelem + kyelem + kzelem

        for ilocal in range(4):
            iglobal = IENelem[ilocal]

            for jlocal in range(4):
                jglobal = IENelem[jlocal]

                K[iglobal,jglobal] += kelem[ilocal,jlocal]
                M[iglobal,jglobal] += melem[ilocal,jlocal]

In [11]:
K  = np.zeros((Npoints, Npoints), dtype = float) 
M  = np.zeros((Npoints, Npoints), dtype = float)

IENfunc3DTet(K, M, IEN)

100%|██████████| 70464/70464 [00:03<00:00, 17618.41it/s]


In [12]:
qvec = np.full(Npoints, Q/(rho*cv))

A = K.copy()
b = M@qvec

In [13]:
# Boundary conditions
Tcc = lambda x, y, z: x*x + y*y + z*z

In [14]:
for i in tqdm(cc):
    A[i,:] = 0.0
    A[i,i] = 1
    b[i] = Tcc(X[i], Y[i], Z[i])

100%|██████████| 4322/4322 [00:00<00:00, 34644.72it/s]


In [15]:
A_inv = np.linalg.inv(A)

In [16]:
T_vec = A_inv@b

In [19]:
error = T_vec - Tcc(X, Y, Z)

SRME = np.sqrt(sum(error**2)/(Npoints))
MAE = sum(error)/(Npoints)
Max  = max(error)

print('O erro quadrático médio é: %.3e' % (SRME))
print('O erro absoluto médio é: %.3e' % (MAE))
print('O erro máximo é: %.3e' % (Max))

O erro quadrático médio é: 8.346e-04
O erro absoluto médio é: 1.531e-04
O erro máximo é: 5.037e-03
